In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
import os
TITANIC_PATH = os.path.join("datasets", "titanic")

In [ ]:
import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [ ]:
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
train_data['Survived'].notna().sum()

In [ ]:
y_train_full = train_data['Survived'] 
y_train_full=y_train_full.to_numpy()

In [ ]:
train_data.drop('Survived', inplace=True, axis=1)
train_data.keys()

In [ ]:
set(train_data['Embarked'])

In [ ]:
#making class manually
def numericalLabelling(status):
    status = str(status)
    if status in ['Capt','Col','Major','Dr','Rev']:
        return 0
    elif status in ['Don','Jonkheer','Master', 'Sir']:
        return 1
    elif status in ['Mr']:
        return 2
    elif status in ['Miss', 'Mlle', 'Mrs', 'Ms']:
        return 3
    elif status in ['Lady','Mme', 'Dona','Countess']:
        return 4

train_data['Status']=train_data['Name'].str.extract(' ([A-Za-z]+)\.', expand = False)
train_data['Status']=train_data['Status'].apply(numericalLabelling)
set(train_data['Status'])

test_data['Status']=test_data['Name'].str.extract(' ([A-Za-z]+)\.', expand = False)
test_data['Status']=test_data['Status'].apply(numericalLabelling)
set(test_data['Status'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_data, y_train_full, random_state=42)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()


data = X_train[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked","Status"]] #useful keys
data.Sex= ((data.Sex == "female" )*1) #convert to numerical

data.Age = data.Age.fillna(data.Age.median())
le.fit(data["Embarked"])
data["Embarked"]=le.transform(data["Embarked"])

X_train=data.to_numpy()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_train

In [ ]:
data = X_valid[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked","Status"]] 
data.Sex= ((data.Sex == "female" )*1) #convert to numerical

data.Age = data.Age.fillna(data.Age.median())
data["Embarked"]=le.transform(data["Embarked"])
X_valid=data.to_numpy()
X_valid=scaler.transform(X_valid)
X_valid

In [ ]:
data_test = test_data[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked","Status"]] 
data_test.Sex= ((data_test.Sex == "female" )*1) #convert to numerical

data_test.Age = data_test.Age.fillna(data_test.Age.median())
data_test.Fare = data_test.Fare.fillna(data_test.Fare.median())
data_test["Embarked"]=le.transform(data_test["Embarked"])
X_test=data_test.to_numpy()
X_test=scaler.transform(X_test)
X_test

In [ ]:
is_NaN = data_test.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = data_test[row_has_NaN]


print(rows_with_NaN)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import History
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
print(tf.__version__)

print(keras.__version__)

In [ ]:
def scheduler(epoch):
    initial_lrate = 0.01#dodac 0 #dodac 0
    drop = 0.9#defult 0.5
    epochs_drop = 8.0
    lrate = initial_lrate * np.power(drop, np.floor((1+epoch)/epochs_drop))
    return lrate

root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

from tensorflow.keras.callbacks import TensorBoard 

tensorboard_cb = TensorBoard(run_logdir)

In [ ]:
initializer = tf.keras.initializers.HeNormal(seed=None)
history = History()
model = Sequential()
model.add(Flatten(input_shape=(X_train.shape[1],)))
model.add(Dense(100, kernel_initializer=initializer,use_bias=True))
model.add(BatchNormalization())
model.add(Activation("elu"))
model.add(Dropout(0.5))
model.add(Dense(50, kernel_initializer=initializer,use_bias=True))
model.add(BatchNormalization())
model.add(Activation("elu"))
model.add(Dropout(0.5))
model.add(Dense(20, kernel_initializer=initializer,use_bias=True))
model.add(BatchNormalization())
model.add(Activation("elu"))
model.add(Dropout(0.5))

model.add(Dense(1,activation="sigmoid"))






model.compile(loss="binary_crossentropy",optimizer="Adam", metrics=["accuracy"])
#round(model.optimizer.lr.numpy(), 5)
callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
history,
tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, mode='min'),
tensorboard_cb]
model.summary()

In [ ]:
history = model.fit(X_train, y_train, validation_data= (X_valid, y_valid), batch_size=32,epochs=500,callbacks=callback)

model.save('LastModel')

%reload_ext tensorboard
%tensorboard --logdir=.my_logs --port=6006

In [ ]:
from sklearn import  metrics
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()


metrics.accuracy_score(y_true= y_valid, y_pred= model.predict_classes(X_valid) )

In [ ]:
import matplotlib.pyplot as plt
predictions =model.predict(X_test)
print(np.shape(predictions))
print()
print(predictions.round(0).astype(int).reshape(-1))

df=pd.DataFrame(predictions.round(0).astype(int).reshape(-1), columns=['Survived'])['Survived']

df2 = pd.concat([data_test['PassengerId'], df], axis=1, join="inner")
print(df2)
#plt.show(predictions.round(0).astype(int).reshape(-1))
prediction = df2.to_csv('prediction.csv',index=False)#